In [83]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [84]:
#Carregando o dataset
df = pd.read_excel("https://raw.githubusercontent.com/irvin-s/smd_project/main/dataset/dry_bean_dataset.xls")

In [85]:
#atribuindo os labes para a classe reposta
labels = ["Barbunya", "Bombay", "Cali", "Dermason", "Horoz", "Seker", "Sira"]

#Dividindo a base em treino e teste
X = df.drop("Class", axis=1)
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=123)

#Normalizando os dados
ss = StandardScaler()
X_train_norm = ss.fit_transform(X_train)
X_test_norm = ss.fit_transform(X_test)

#Transformando a variável cartegorica em binária
labelencoder = LabelEncoder()
y_train_bin = labelencoder.fit_transform(y_train)
y_test_bin = labelencoder.fit_transform(y_test)

In [86]:
#Aplicando o modelo KNN

#Definindo o valor de vizinhos
classifier = KNeighborsClassifier(n_neighbors=15)

#Treinar o modelo, com os dados de treinamento
classifier.fit(X_train_norm, y_train_bin)

#Prever os valores de y com dos dados de X_test
y_pred = classifier.predict(X_test_norm)

# Imprimindo a matriz confusa
print("Matriz Confusa KNN: ")
print(confusion_matrix(y_test_bin, y_pred), "\n")  

# Imprimindo o relatório de classificação
print("Relatório de classificação KNN: \n", classification_report(y_test_bin, y_pred, target_names=labels))  

# Imprimindo o quão acurado foi o modelo
acu_knn = accuracy_score(y_test_bin, y_pred) * 100
print("Acurácia KNN: {:.2f}%".format(acu_knn))

Matriz Confusa KNN: 
[[ 384    0   34    0    2    3   13]
 [   0  171    0    0    0    0    0]
 [  11    0  507    0    5    2    7]
 [   0    0    0 1064    0   12   68]
 [   0    0   14    2  620    0   18]
 [   2    0    0   19    1  628   24]
 [   1    0    0   93    9    6  772]] 

Relatório de classificação KNN: 
               precision    recall  f1-score   support

    Barbunya       0.96      0.88      0.92       436
      Bombay       1.00      1.00      1.00       171
        Cali       0.91      0.95      0.93       532
    Dermason       0.90      0.93      0.92      1144
       Horoz       0.97      0.95      0.96       654
       Seker       0.96      0.93      0.95       674
        Sira       0.86      0.88      0.87       881

    accuracy                           0.92      4492
   macro avg       0.94      0.93      0.93      4492
weighted avg       0.92      0.92      0.92      4492

Acurácia KNN: 92.30%


In [87]:
#Aplicando a Árvore de decisão

#Instanciando o modelo
model = DecisionTreeClassifier()

# Treinar o modelo
model.fit(X_train, y_train_bin)

# Aplicar o modelo ao treinamento e ao teste
predicted_test_y = model.predict(X_test)

predicted_train_y = model.predict(X_train)

In [88]:
# Imprimindo a matriz confusa
print("Matriz Confusa Decision Tree: ")
print(confusion_matrix(y_test_bin, predicted_test_y), "\n")  

# Imprimindo o relatório de classificação
print("Relatório de classificação Decision Tree: \n", classification_report(y_test_bin, predicted_test_y, target_names=labels)) 

#Imprimindo a acurácia do modelo
accuracy_dt = accuracy_score(y_test_bin, predicted_test_y) * 100
print("Acurácia Decision Tree: {:.2f}%.".format(accuracy_dt))

Matriz Confusa Decision Tree: 
[[ 381    0   35    0    6    3   11]
 [   0  171    0    0    0    0    0]
 [  27    0  487    0   11    2    5]
 [   0    0    0 1038    3   12   91]
 [   2    0   14    7  609    0   22]
 [   5    0    1   31    0  615   22]
 [   7    0    1  100   17   16  740]] 

Relatório de classificação Decision Tree: 
               precision    recall  f1-score   support

    Barbunya       0.90      0.87      0.89       436
      Bombay       1.00      1.00      1.00       171
        Cali       0.91      0.92      0.91       532
    Dermason       0.88      0.91      0.89      1144
       Horoz       0.94      0.93      0.94       654
       Seker       0.95      0.91      0.93       674
        Sira       0.83      0.84      0.84       881

    accuracy                           0.90      4492
   macro avg       0.92      0.91      0.91      4492
weighted avg       0.90      0.90      0.90      4492

Acurácia Decision Tree: 89.96%.


In [89]:
#Aplicando a Random Forest

#Instanciando o modelo
rf = RandomForestClassifier(90, max_depth=10, random_state=42)
rf.fit(X_train,y_train_bin)

#Aplicando o modelo
y_pred_rf = rf.predict(X_test)

# Imprimindo a matriz confusa
print("Matriz Confusa Random Forest: ")
print(confusion_matrix(y_test_bin, y_pred_rf), "\n")  

# Imprimindo o relatório de classificação
print("Relatório de classificação Random Forest: \n", classification_report(y_test_bin, y_pred_rf, target_names=labels)) 

#Imprimindo a acurácia do modelo
accuracy_rf = accuracy_score(y_test_bin, y_pred_rf) * 100
print("Acurácia Random Forest: {:.2f}%.".format(accuracy_rf))

Matriz Confusa Random Forest: 
[[ 381    0   36    0    4    3   12]
 [   0  171    0    0    0    0    0]
 [  13    0  505    0    5    2    7]
 [   0    0    0 1080    0   10   54]
 [   0    0   15    2  621    0   16]
 [   2    0    0   25    0  627   20]
 [   2    0    1   94   15    8  761]] 

Relatório de classificação Random Forest: 
               precision    recall  f1-score   support

    Barbunya       0.96      0.87      0.91       436
      Bombay       1.00      1.00      1.00       171
        Cali       0.91      0.95      0.93       532
    Dermason       0.90      0.94      0.92      1144
       Horoz       0.96      0.95      0.96       654
       Seker       0.96      0.93      0.95       674
        Sira       0.87      0.86      0.87       881

    accuracy                           0.92      4492
   macro avg       0.94      0.93      0.93      4492
weighted avg       0.92      0.92      0.92      4492

Acurácia Random Forest: 92.30%.


In [90]:
#Aplicando a Rede Neural MLP

#Instanciado o modelo
modelNN = Sequential()
modelNN.add(Dense(256, input_shape=(16,), activation="sigmoid"))
modelNN.add(Dense(64, activation="sigmoid"))
modelNN.add(Dense(7, activation="softmax"))

#Vetorizar a classe resposta
lb = LabelBinarizer()
y_train_vet = lb.fit_transform(y_train)
y_test_vet = lb.fit_transform(y_test)

#Realizando o treinamento
modelNN.compile(optimizer=SGD(0.01), loss="categorical_crossentropy", metrics=["accuracy"])
H = modelNN.fit(X_train_norm, y_train_vet, batch_size=128, epochs=10, verbose=2, validation_data=(X_test_norm, y_test_vet))

#Avalidando a Rede Neural 
predictions = modelNN.predict(X_test_norm, batch_size=128)

# Imprimindo a matriz confusa
print("Matriz Confusa Rede Neural MLP: ")
print(confusion_matrix(y_test_vet.argmax(axis=1), predictions.argmax(axis=1)), "\n")  

# Imprimindo o relatório de classificação
print("Relatório de classificação Rede Neural MLP: \n", classification_report(y_test_vet.argmax(axis=1), predictions.argmax(axis=1), target_names=labels, zero_division=0))

#Imprimindo a acurácia do modelo
accuracy_nn = accuracy_score(y_test_vet.argmax(axis=1), predictions.argmax(axis=1)) * 100
print("Acurácia Rede Neural MLP: {:.2f}%.".format(accuracy_nn))

#Matriz confusa
#cnf_matrix = confusion_matrix(X_test_norm, predictions.argmax(axis=1))

Epoch 1/10
72/72 - 1s - loss: 1.8566 - accuracy: 0.2301 - val_loss: 1.8246 - val_accuracy: 0.2547
Epoch 2/10
72/72 - 0s - loss: 1.8084 - accuracy: 0.2634 - val_loss: 1.7985 - val_accuracy: 0.2547
Epoch 3/10
72/72 - 0s - loss: 1.7848 - accuracy: 0.2634 - val_loss: 1.7757 - val_accuracy: 0.2547
Epoch 4/10
72/72 - 0s - loss: 1.7614 - accuracy: 0.2634 - val_loss: 1.7528 - val_accuracy: 0.2547
Epoch 5/10
72/72 - 0s - loss: 1.7371 - accuracy: 0.2634 - val_loss: 1.7272 - val_accuracy: 0.2547
Epoch 6/10
72/72 - 0s - loss: 1.7113 - accuracy: 0.2634 - val_loss: 1.7005 - val_accuracy: 0.2551
Epoch 7/10
72/72 - 0s - loss: 1.6836 - accuracy: 0.2652 - val_loss: 1.6732 - val_accuracy: 0.2553
Epoch 8/10
72/72 - 0s - loss: 1.6541 - accuracy: 0.2750 - val_loss: 1.6410 - val_accuracy: 0.2845
Epoch 9/10
72/72 - 0s - loss: 1.6216 - accuracy: 0.3117 - val_loss: 1.6108 - val_accuracy: 0.2763
Epoch 10/10
72/72 - 0s - loss: 1.5879 - accuracy: 0.3249 - val_loss: 1.5730 - val_accuracy: 0.3689
Matriz Confusa Rede

In [ ]:
#Ensamble de Redes Neurais


In [81]:
#Ensamble heterogêneo

#Instanciando o modelo
rfr = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Treinando o modelo no dataset de treino
rfr.fit(X_train, y_train_vet)

#Avaliando o desempenho através do erro médio absoluto
#score = -1*cross_val_score(rfr, X_test, y_test_bin, cv = 10, scoring = 'neg_mean_absolute_error').mean()

#print(score)
#Aplicando o modelo
y_pred_en = rfr.predict(X_test)

#Calcula o erro absoluto
errors = abs(y_pred_en - y_test_vet)

#Calcula a porcentagem do erro absoluto medio(MAPE)
mape = 100 * (errors / y_test_vet)

# Imprimindo a matriz confusa
#print("Matriz Confusa Ensamble heterogêneo: ")
#print(confusion_matrix(y_test_bin, y_pred_en, "\n"))  

# Imprimindo o relatório de classificação
#print("Relatório de classificação Ensamble heterogêneo: \n", classification_report(y_test_bin, y_pred_en, target_names=labels))

#Imprimindo a acurácia do modelo
#accuracy_en = accuracy_score(y_test_bin, y_pred_en) * 100
#print("Acurácia Ensamble heterogêneo: {:.2f}%.".format(accuracy_en))
accuracy_en = 100 - np.mean(mape)
print('Acurácia Ensamble heterogêneo:', round(accuracy_en, 2), '%.')

Acurácia Ensamble heterogêneo: nan %.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
